#  Проект "Анализ базы данных книг"

### Цель исследования:
Проанализировать предоставленые данные, сформировать ценностное предложение для сервиса чтения электронных книг

### Описание данных
Таблица **books** содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя. 

Таблица **authors** содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора. 

Таблица **publishers** содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства; 

Таблица **ratings** содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.
Таблица **reviews** содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

### Задачи
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
#загружаем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

In [14]:
# Подключимся к базе
db_config = {} # пароли и адреса удалены 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

***Рассмотрим подробнее каждую таблицу, для этого выведем первые 5 строк каждой из них***

In [15]:
# рассмотрим первые строки датасета books
pd.io.sql.read_sql('''select * from books limit 5''', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [16]:
# выведем общую информацию
pd.io.sql.read_sql('''select * from books ''', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [17]:
# выведем первые строки датасета autors
pd.io.sql.read_sql('''select * from authors limit 5''', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [18]:
# аналогично выведем общую информацию
pd.io.sql.read_sql('''select * from authors ''', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [19]:
# выведем первые строки датасета publishers
pd.io.sql.read_sql('''select * from publishers limit 5''', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [20]:
# выведем общую инфоормацию
pd.io.sql.read_sql('''select * from publishers ''', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [21]:
# выведем первые строки датасета ratings
pd.io.sql.read_sql('''select * from ratings limit 5''', con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [22]:
# выведем общую инфоормацию
pd.io.sql.read_sql('''select * from ratings ''', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [23]:
# выведем первые строки датасета reviews
pd.io.sql.read_sql('''select * from reviews limit 5''', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [24]:
# выведем общую инфоормацию
pd.io.sql.read_sql('''select * from reviews ''', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


***Мы вывели общую информацию о датасетах и имеем представление о  размерности и содержимом датасетов***

**Задача 1.** 

Посчитайте, сколько книг вышло после 1 января 2000 года;

In [25]:
query = 'select count(book_id) from books where to_char(publication_date,yyyy-mm-dd) >= \'2000-01-01\''
df = pd.io.sql.read_sql('''select count(book_id) 
                            from books 
                            where publication_date >= \'2000-01-01\''''
                        , con = engine)
print('После 1 января 2000 года вышло', df['count'].sum(), 'книг')

После 1 января 2000 года вышло 821 книг


**Задача 2.** 

Для каждой книги посчитайте количество обзоров и среднюю оценку

Выберем из книг самые читаемые и обсуждаемые

In [26]:
pd.io.sql.read_sql('''
                with book_rev as (
                select
                    book_id
                    ,count(review_id) as review_count
                from  reviews
                group by book_id
                )
                ,
                book_rg as (
                select 
                    book_id
                    ,avg(rating) as avg_rating
                from ratings
                group by book_id
                 )
                 select 
                     b.title
                     ,brg.book_id
                     ,brv.review_count
                     ,brg.avg_rating
                 from books b
                     left join book_rg brg
                         on b.book_id = brg.book_id
                     left join book_rev brv
                         on b.book_id = brv.book_id
                group by b.title
                     ,brg.book_id
                     ,brv.review_count
                     ,brg.avg_rating
                order by brv.review_count desc
                        ,brg.avg_rating desc
                
            
            ''', con = engine)

,title,book_id,review_count,avg_rating
0,The Cat in the Hat and Other Dr. Seuss Favorites,672,NaN,5.000000
1,Disney's Beauty and the Beast (A Little Golden...,191,NaN,4.000000
2,Essential Tales and Poems,221,NaN,4.000000
3,Leonardo's Notebooks,387,NaN,4.000000
4,Anne Rice's The Vampire Lestat: A Graphic Novel,83,NaN,3.666667
...,...,...,...,...
995,Debt of Honor (Jack Ryan #7),187,1.0,3.000000
996,Moo Baa La La La!,446,1.0,3.000000
997,1 000 Places to See Before You Die,2,1.0,2.500000
998,History of Beauty,318,1.0,2.500000


В этой таблице мы можем увидеть рейтинг самых читаемых и обсуждаемых книг. Как видно из нее, не всегда книги с высоким рейтингом обсуждают. 


**Задача 3.** 

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [27]:
# Найдем в данных издательство, которое выпустило наибольшее число книг
#(исключив при этом тонкие книги с количеством страниц меньше 50)
pd.io.sql.read_sql('''select
                        pl.publisher
                        ,count(b.book_id)
                    from  books b
                        join publishers pl
                            on pl.publisher_id  = b.publisher_id  
                    where b.num_pages  >= 50 
                    group by pl.publisher
                    order by count(b.book_id) desc
                    limit 1        
                    
                    ''', con = engine)

,publisher,count
0,Penguin Books,42


Издательство "Penguin Books" выпустило 42 книги. Имеет смысл обратить внимание на книги изданные этим издательством.

In [28]:
pd.io.sql.read_sql('''select
                        pl.publisher
                        ,b.title
                        ,avg(rg.rating)
                    from  books b
                        join publishers pl
                            on pl.publisher_id  = b.publisher_id
                        join ratings rg
                            on rg.book_id = b.book_id
                    where b.num_pages  >= 50 and  pl.publisher = 'Penguin Books'
                    group by pl.publisher
                                ,b.title
                    order by avg(rg.rating) desc
                    --limit 1        
                    
                    ''', con = engine)

,publisher,title,avg
0,Penguin Books,On Beauty,4.500000
1,Penguin Books,A Princess of Mars (Barsoom #1),4.500000
2,Penguin Books,The Body,4.500000
3,Penguin Books,Icy Sparks,4.500000
4,Penguin Books,Lost in a Good Book (Thursday Next #2),4.500000
5,Penguin Books,Far From the Madding Crowd,4.250000
6,Penguin Books,The Eyre Affair (Thursday Next #1),4.250000
7,Penguin Books,Emma,4.150000
8,Penguin Books,Alexander Hamilton,4.000000
9,Penguin Books,Disgrace,4.000000


В издательстве "Penguin Books" не все книги с высоким рейтингом. Мы вывели рейтинг каждой книги,теперь видно на какие книги этого издательства стоит обратить внимание.

**Задача 4.** 

Определите автора с самой высокой средней оценкой книг

In [29]:
# Найдем автора с самой высокой средней оценкой книг. Выбирать будем из книг, у которых 50 и более оценок.
pd.io.sql.read_sql('''
                    with book_rg as(
                    select
                        book_id
                        ,avg(rating) as avg_raiting 
                    from   ratings rg    
                    group by book_id
                    having count(rating_id) >= 50
                    )
                    select 
                        au.author 
                        ,b.author_id
                        ,avg(br.avg_raiting)
                    from book_rg br
                        left join books b
                            on br.book_id = b.book_id
                        left join authors au
                            on b.author_id = au.author_id
                    group by au.author
                            ,b.author_id
                    order by avg(br.avg_raiting) desc
                    
                    
                    ''', con = engine)

,author,author_id,avg
0,J.K. Rowling/Mary GrandPré,236,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,402,4.264151
2,J.R.R. Tolkien,240,4.258446
3,Louisa May Alcott,376,4.192308
4,Rick Riordan,498,4.080645
5,William Golding,621,3.901408
6,J.D. Salinger,235,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,469,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,630,3.787879
9,Dan Brown,106,3.754540



Неудивительно, что автор с самым высоким рейтингом Джоан Роулинг. Ее книги читают и взрослые и дети.


In [30]:
pd.io.sql.read_sql('''select
                        au.author
                        ,b.title
                        ,avg(rg.rating)
                    from  books b
                        join ratings rg
                            on rg.book_id = b.book_id 
                        join authors au
                            on au.author_id = b.author_id     
                    where au.author='J.K. Rowling/Mary GrandPré'
                    group by au.author
                            ,b.title                    
                    ''', con = engine)

,author,title,avg
0,J.K. Rowling/Mary GrandPré,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.500000
1,J.K. Rowling/Mary GrandPré,Harry Potter and the Chamber of Secrets (Harry...,4.287500
2,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...,4.246575
3,J.K. Rowling/Mary GrandPré,Harry Potter and the Order of the Phoenix (Har...,4.186667
4,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634


Мы вывели книги самого читаемого в данный момент автора. Почти у всех хороший рейтинг. Имеет смысл включить их в перечень рекомендованных книг для приложения. 

**Задача 5.** 

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [31]:
pd.io.sql.read_sql('''
                    with username as (
                    select
                        rg.username 
                    from   ratings rg
                    group by rg.username
                    having count(rg.rating_id) > 50
                    ),
                    mean_count as(
                    select
                         rv.username
                         ,count(rv.review_id) as count_rev
                    from username un 
                    join reviews rv
                            on rv.username  = un.username 
                    group by rv.username 
                    
                    )
                    select avg(count_rev)
                    from mean_count
                          
                    
                    ''', con = engine)

,avg
0,24.333333


В среднем пользователи , которые ставят больше 50 оценок, пишут более 24 отзывов в целом. Думаю, таких пользователй нужно привлекать больше, поскольку они читают много и охотно делятся информацией по книгам. 

## Общие выводы:

1. Мы вывели ТОП самых обсуждаемых книг. Стоит обратить внимание на них для привлечения пользователей.

2. Издательство **Penguin Books** издает наибольшее количество книг, многие из них с неплохим рейтингом. 

3. Наиболее популярный автор в данный момент это Джоан Роулинг, написавшая "Гарри Поттера". Среди ее книг много других популярных. Это стоит использовать для привлечения аудитории.

4. Среди пользователей среднее количество отзывов более 24.  Это значит они много читают и много делятся инфомацией о книгах. Стоит привлечь таких пользователей. Возможно, запустить маркетинговую акцию, с определенными бонусами за отзыв о прочитанной книге. 